In [ ]:
import os
from pathlib import Path
import calendar

import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

import requests as rq

# for aWhere API
from header import AWhereAPI
from secret_codes import *

api_key = API_KEY
api_secret = API_SECRET

In [ ]:
# Create aWhere object
aw = AWhereAPI(api_key, api_secret)

# To get auth token, encode secret and key
sc = aw.encode_secret_and_key(API_KEY, API_SECRET)

# Then call auth token
token = aw.get_oauth_token(sc)

In [ ]:
# Set variables for aWhere API call 

# General endpoint
endpt = "https://api.awhere.com"

# Set up headers: auth and specific header for agronomics data
auth_headers = {"Authorization": "Bearer %s" % token,
                "Content-Type": 'application/json'}

agro_header = {"Authorization":  "Bearer %s" % token}

In [ ]:
# Load area of interest into gpd

# Create path to home
home = str(Path.home())

# Create path to Siskiyou geojson since csv crashes Jupyter
path_to_josephine = os.path.join(
    home, "Desktop", "drought-tracker", "data", "TIGER", "josephine.geojson")

# Create df with 1 county (Siskiyou)
josephine = gpd.read_file(path_to_josephine)

# Convert into CRS
josephine = josephine.to_crs("EPSG:4236")

In [ ]:
def create_awhere_grid(aoi, out_crs, calc_crs):
    """
    Parameters
    ----------
    aoi: Geopandas GeoDataFrame
        One-element GeoDataFrame containing area over which
        to draw grid
    
    out_crs: numeric string
        String of EPSG code for exported GDF
    
    calc_crs: numeric string
        String of EPSG code for CRS used
        to calculate grid. CRS must have units of meters
        to work with aWhere's API
        
    Returns
    -------
    GeoDataFrame containing 9km x 9km cells
    """   
    
    # Reproject aoi to CRS using meters for aWhere grid cells
    aoi = aoi.to_crs(f'EPSG:{calc_crs}')
    
    # Get x and y min and max from total boundaries
    xmin, ymin, xmax, ymax = aoi.total_bounds
    
    # Set side of grid cell to 9 km
    side = 9000
    
    # Create x values for x points for rows
    x_range = np.arange(int(np.floor(xmin)), int(np.ceil(xmax)), side)
    
    # Create y values
    y_range = np.arange(int(np.floor(ymin)), int(np.floor(ymax)), side)
    
    # Create empty list to hold grid cells
    polygons = []
    
    for x in x_range:
        for y in y_range:
            polygons.append(Polygon([(x,y), (x+side, y), (x+side, y+side), (x,y+side)]))
            
    grid = gpd.GeoDataFrame({'geometry': polygons}, crs=f"EPSG:{calc_crs}")
    
    #grid.set_crs = f'EPSG:{calc_crs}'
    
    # Convert to out_crs
    grid = grid.to_crs(f"EPSG:{out_crs}")
    
    return grid

In [ ]:
# Create grid polygon
grid = create_awhere_grid(aoi=josephine,
                          out_crs='4236',
                          calc_crs='2019')

# Add centroid column
grid['centroid'] = grid.geometry.apply(lambda poly: poly.centroid)

# Check to see if everything looks correct
fig, ax = plt.subplots()
grid.plot(color="none", linewidth=1, edgecolor='orange', ax=ax, zorder=3)
josephine.plot(color='green', zorder=2, ax=ax)

plt.show()

In [ ]:
def call_api(lat, lon, start_date, end_date, testing=False):
    
        # URL to historical agronomics (norms)
        hist_ag_url = f"/v2/agronomics/locations/{lat},{lon}/agronomicvalues/{start_date},{end_date}"

        # Endpoint suffix to get all accumulations
        url_append = "?properties=accumulations"

        # Full URL, endpt is defined up top
        full_url = endpt + hist_ag_url + url_append
        
        if testing == False:
        
            # Get JSON
            ag_norms_json = rq.get(full_url, headers=agro_header).json()
            
            return ag_norms_json
            
        elif testing == True:
            
            print(full_url)

In [ ]:
def get_ppet(grid_df, start_date, end_date, drop_null=False, testing=False):
    """
    Paraemeters
    -----------
    grid_df: Geopandas GeoDataFrame
        Contains gridded area to fetch argonomics values for

    start_date: string
        Format "YYYY-MM-DD"

    end_date: string
        Format "YYYY-MM-DD"

    drop_null: boolean (optional)
        Drop grid cells that have null values returned from
        API request for agronomics data. Often null values 
        result from grid cell lying in a body of water.

    testing: boolean (optional)
        Returns sample P/PET values for each grid cell 
        from 0 - 30 mm. Used for testing; bypasses API call.

    Returns
    -------
    grid_df: Geopandas GeoDataFrame
        Contains gridded input area with requested values.

    """

    # For testing, generate random values for each cell
    if testing == True:
        grid_df['test_ppet'] = [np.random.randint(
            0, 30) for i in range(0, grid.shape[0])]

        return grid_df

    # Add P/PET column if it does not exist and initialize to 0
    if 'ppet' not in grid_df.columns:
        grid_df = grid_df.assign(ppet=0)

    # Iterate thru rows (cells) in gdf
    for index, row in grid_df.iterrows():
        
        #Print progress
        print(f"On row {index + 1} of {grid.shape[0]}")
        
        # Get centroid coordinates from each cell to pass to API
        lat = round(row.geometry.centroid.y, 5)
        lon = round(row.geometry.centroid.x, 5)

        # Get JSON using call_api function
        ag_norms_json = call_api(lat, lon, start_date, end_date)

        # Try to pull data from return JSON
        try:
            
            vals = ag_norms_json.get('accumulations').get('ppet')
            
            # Testing
            if len(vals) == 0:
                print(f"Vals not returned for object number {row}")
            
            grid_df.loc[index, 'ppet'] = vals 

        except:
            grid_df.loc[index, 'ppet'] = np.nan

        # Drop null cells if requested
        if drop_null == True:
            grid_df = grid_df[grid_df.ppet.isnull() == False]

    return grid_df

In [ ]:
ppet_grid = get_ppet(grid_df=grid,
                     start_date="2020-04-01",
                     end_date="2020-04-30",
                     drop_null=False,
                     testing=False)

In [ ]:
"""Add day range to this eventually"""
def binary_threshold(gdf, daily_acc_ppet_threshold, month=None, year=None):

    # Find monthly accumulated P/PET
    #days_in_month = calendar.monthrange(year, month)[1]

    # Calculate monthly accumulated P/PET based on user's threshold
    #monthly_acc_ppet = days_in_month * daily_acc_ppet_threshold
    monthly_acc_ppet = 30 * daily_acc_ppet_threshold

    # Add Boolean flag column if meets threshold
    gdf['meet_thresh'] = gdf['ppet'].apply(
        lambda x: 1 if x <= monthly_acc_ppet else 0)

    return gdf

In [ ]:
binary_grid = binary_threshold(gdf=ppet_grid,
                               daily_acc_ppet_threshold=0.2,
                               month=4,
                               year=2020)

In [ ]:
def to_convex_hull(gdf, crs):
    
    # Drop all rows that do not meet threshold
    meet_thresh_only = gdf[gdf.meet_thresh == 1]
    
    # Dissolve polygons that meet threshold
    meet_thresh_dissolve = meet_thresh_only.dissolve(by='meet_thresh')
    
    # Explode polygons so that there are unique contiguous areas
    exploded = meet_thresh_dissolve.explode()

    # Drop multi-index outer level generated from dissolve
    exploded = exploded.droplevel(0)
    
    # Get largest polygon
    largest = exploded[exploded.area == exploded.area.max()]
    
    # Generate convex hull
    hull_largest = largest.convex_hull

    # Convert to gdf
    hull = gpd.GeoDataFrame(hull_largest, crs=gdf.crs)

    # Rename column '0' to 'geometry'
    hull.rename(columns={0:"geometry"}, inplace=True)
    
    return hull

In [ ]:
hull = to_convex_hull(binary_grid, 4326)

In [ ]:
fig, ax = plt.subplots()

hull.plot(ax=ax, color='none', edgecolor='red', zorder=4)
binary_grid.plot(column='ppet', cmap='coolwarm_r', ax=ax, alpha=0.6, zorder=3)
josephine.plot(color='green', alpha=0.9, ax=ax)

plt.show()

In [ ]:
def proportional_split(grid_gdf, hull_gdf, crs):
    
    grid_gdf = grid_gdf.to_crs(f"epsg:{crs}")
    
    hull_gdf = hull_gdf.to_crs(f"epsg:{crs}")
    
    # Set summary variables to 0
    inter_area_run_sum = 0
    
    # This is a list to more easily calculate average (using sum then len)
    inter_ppet_run_sum = []
    
    for index, row in grid_gdf.iterrows():
        
        if row.geometry.intersects(hull_gdf.geometry[0]):
            
            intersect_area = row.geometry.intersection(hull_gdf.geometry[0]).area
            
            # Find the proportional split of grid cell's P/PET
            proportional_ppet = (intersect_area / row.geometry.area) * row.ppet
            
            # Append it to the list
            inter_ppet_run_sum.append(proportional_ppet)
    
    # Find average P/PET per cell
    inter_ppet = round((sum(inter_ppet_run_sum) / len(inter_ppet_run_sum)), 3)
    
    print(f"Total droughted area: {hull_gdf.geometry[0].area}")

    print(f"Average P/PET in droughted area: {inter_ppet}")
    
proportional_split(binary_grid, hull, "2019")

In [ ]:
start_year = 2010
end_year = 2012
start_day = "04-01"
end_day = "04-30"

# Calculate range of years
years_range = np.arange(start_year, end_year + 1)

# Create grid (does not change)
grid = create_awhere_grid(aoi=josephine,
                          out_crs='4236',
                          calc_crs='2019')

# Hold returned dfs for testing
hulls_list = []

for year in years_range:
    # get grid
    year_grid = grid

    # Add P/PET values to entire grid
    # Finctions calls API on centroid of each grid cell
    year_grid_ppet = get_ppet(grid_df=year_grid,
                              start_date="2020-04-01",
                              end_date="2020-04-30",
                              drop_null=False,
                              testing=False)

    # Reclassify grid using binary thresholding (1 = over threshold)
    year_binary_grid = binary_threshold(gdf=year_grid_ppet,
                                        daily_acc_ppet_threshold=0.2)
    # Generate convex hull
    year_convex_hull = to_convex_hull(year_binary_grid, 4326)
    
    hulls_list.append(year_convex_hull)
    # (proportional split)
    # add convex_hull, proportional_split as tuple

# create gdf from returned_data with idx = year, geom=convex_hull, data=proportional_split
# find centroids of hulls
# plot centroids

In [ ]:
hulls_list[2]